<a href="https://colab.research.google.com/github/lemarakis/fastmcp/blob/master/MCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q fastmcp google-generativeai

In [2]:
#User question
#user_question = ("πόση μνήμη χρησιμοποιεί ο server αυτή τη στιγμή, πόση CPU και πόσος χώρος υπάρχει στο /tmp?")
#user_question = ("πόση ώρα τρέχει ο server?")
#user_question = ("δώσε μου στοιχεία για τον nginx")
#user_question = ("Καλημέρα ! Πως σε λένε?")
user_question = "πόσο ελεύθερο χώρο έχω στο /tmp και πόση RAM καταναλώνω αυτή την στιγμή"
print(user_question)

πόσο ελεύθερο χώρο έχω στο /tmp και πόση RAM καταναλώνω αυτή την στιγμή


In [3]:
from fastmcp import Client
from google.colab import userdata
import google.generativeai as genai
import json

# ---------------------------
# Gemini setup
# ---------------------------

genai.configure(api_key=userdata.get("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash")

# ---------------------------
# MCP server
# ---------------------------

MCP_SERVER_URL = "http://195.251.254.31:8000/sse"

# ---------------------------
# Agent logic
# ---------------------------

async def main():

    async with Client(MCP_SERVER_URL) as mcp:

        # 1. Dynamic tool discovery
        tools = await mcp.list_tools()

        tool_descriptions = ""
        for t in tools:
            tool_descriptions += f"- {t.name}: {t.description}\n"


        # 2. Tool + arguments planning (1st LLM call)
        planning_prompt = f"""
Είσαι ένας agent που μπορεί να χρησιμοποιεί εργαλεία.

Διαθέσιμα εργαλεία:
{tool_descriptions}

Ερώτηση χρήστη:
"{user_question}"

ΟΔΗΓΙΕΣ (ΥΠΟΧΡΕΩΤΙΚΟ):
- Απάντησε ΜΟΝΟ με καθαρό JSON
- ΜΗΝ χρησιμοποιήσεις markdown
- ΜΗΝ προσθέσεις ```json ή ```
- ΜΗΝ προσθέσεις κείμενο πριν ή μετά
- Η απάντηση ΠΡΕΠΕΙ να ξεκινά με [ και να τελειώνει με ]

Μορφή απάντησης:
[
  {{
    "tool": "<tool_name>",
    "arguments": {{ "<arg>": "<value>" }}
  }}
]
"""

        llm_response = model.generate_content(planning_prompt)
        raw_text = llm_response.text.strip()

        print("RAW LLM OUTPUT:", repr(raw_text))

        plan = json.loads(raw_text)

        print("Agent plan:", plan)

        # 3. Execute MCP tools (tool-agnostic)
        results = {}
        for step in plan:
            tool_name = step["tool"]
            arguments = step.get("arguments", {})
            results[tool_name] = await mcp.call_tool(tool_name, arguments)

        # 4. Final answer (2nd LLM call)
        final_prompt = f"""
Ο χρήστης ρώτησε:
"{user_question}"

Δεδομένα από MCP εργαλεία:
{results}

Δώσε μία σύντομη και κατανοητή απάντηση στα Ελληνικά.
"""

        answer = model.generate_content(final_prompt)

        print("\nΤελική απάντηση:")
        print(answer.text)

await main()


/usr/local/lib/python3.12/dist-packages/google/colab/html/_background_server.py:103: DeprecationWarning: make_current is deprecated; start the event loop first
  ioloop.make_current()
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


RAW LLM OUTPUT: '[\n  {\n    "tool": "get_disk_usage",\n    "arguments": {\n      "path": "/tmp"\n    }\n  },\n  {\n    "tool": "get_memory_usage",\n    "arguments": {}\n  }\n]'
Agent plan: [{'tool': 'get_disk_usage', 'arguments': {'path': '/tmp'}}, {'tool': 'get_memory_usage', 'arguments': {}}]

Τελική απάντηση:
Έχετε 31.4 GB ελεύθερο χώρο στο /tmp.
Αυτή τη στιγμή καταναλώνετε 4.48 GB RAM, που αντιστοιχεί στο 7.1% της συνολικής.
